In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import (GridSearchCV, KFold, cross_val_score,
                                     train_test_split)
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

%matplotlib inline

# データの読み込み

In [2]:
df = pd.read_csv("train.csv") # データをダウンロード
X = df[['GrLivArea', 'YearBuilt']] # 2つの特徴量を抜き出し変数に格納
y = df[['SalePrice']] # 目的変数を抜き出し、変数に格納
X.shape

(1460, 2)

# train_test_split

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=0.8)

# 対数変換

In [4]:
#対数変換処理をする
X_train_log = np.log(X_train)
y_train_log = np.log(y_train)
X_test_log = np.log(X_test)
y_test_log = np.log(y_test)

 # 【問題1】ブレンディングのスクラッチ実装
ブレンディング をスクラッチ実装し、単一モデルより精度があがる例を 最低3つ 示してください。精度があがるとは、検証用データに対する平均二乗誤差（MSE）が小さくなることを指します。

## 学習・推定

In [5]:
#SVM
svm = SVR(gamma='scale', C=1.0, epsilon=0.2) # SVCに訓練データを学習させる
svm.fit(X_train_log, y_train_log.values.ravel()) # SVCによる検証データの分類
svm_predict = svm.predict(X_test_log) #推定
svm_mse = mean_squared_error(y_test_log, svm_predict) #mse

In [6]:
# 決定木
dtr = DecisionTreeRegressor(max_depth=3)
dtr.fit(X_train_log, y_train_log)#学習
dtr_predict = dtr.predict(X_test_log)#推定
dtr_mse = mean_squared_error(y_test_log, dtr_predict) #mse

In [7]:
#線形回帰
lr = LinearRegression()
lr.fit(X_train_log, y_train_log)#学習
lr_predict = lr.predict(X_test_log)#推定
lr_mse = mean_squared_error(y_test_log, lr_predict) #mse

In [8]:
#単一モデルの精度
score = pd.DataFrame([[svm_mse], [dtr_mse], [lr_mse]], columns={'単一モデルのMSE'}, index=['SVM', '決定木', '線形回帰'])
score

,単一モデルのMSE
SVM,0.052578
決定木,0.071384
線形回帰,0.044795


## 1. 平均をとる

In [9]:
#単一モデルの精度結果をもとに平均をとる
blend_1 = (svm_predict.reshape(-1, 1)+ dtr_predict.reshape(-1, 1) + lr_predict.reshape(-1, 1))/3
#MSE
blend_1_mse = mean_squared_error(y_test_log, blend_1) #mse
print('blend_1_mse',blend_1_mse)

blend_1_mse 0.05054710577445747


## 2. 各モデルに重みを付ける

In [10]:
blend_2 = (svm_predict.reshape(-1, 1)*0.5 + dtr_predict.reshape(-1,1)*0.3 + lr_predict.reshape(-1,1)*0.2)
#mse
blend_2_mse = mean_squared_error(y_test_log, blend_2)
print('blend_2_mse',blend_2_mse)

blend_2_mse 0.051326980571388625


## 3.SVMのカーネルの変更

In [11]:
#kernel='poly'
svm_1 = SVR(kernel='poly', gamma='auto') # SVCに訓練データを学習させる
svm_1.fit(X_train_log, y_train_log.values.ravel()) # SVCによる検証データの分類
svm_1_predict = svm_1.predict(X_test_log) #推定

In [12]:
#kernel='sigmoid'
svm_2 = SVR(kernel='sigmoid', gamma='auto') # SVCに訓練データを学習させる
svm_2.fit(X_train_log, y_train_log.values.ravel()) # SVCによる検証データの分類
svm_2_predict = svm_2.predict(X_test_log) #推定

In [13]:
blend_3 = (svm_1_predict.reshape(-1, 1) + svm_2_predict.reshape(-1, 1) + svm_predict.reshape(-1, 1))/3
#mse
blend_3_mse = mean_squared_error(y_test_log, blend_3)
print('blend_3_mse', blend_3_mse)

blend_3_mse 0.061561642606966886


## 4. SVMにグリッドサーチをかけ,パラメーターチューニングを行う。

In [14]:
y_train_log.shape

(1168, 1)

In [15]:
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svr = SVR(gamma="scale")
gs = GridSearchCV(svr, parameters, cv=5)
gs.fit(X_train_log, y_train_log.values.ravel())

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [16]:
#結果の出力
print('Test set score: {}'.format(gs.score(X_test_log, y_test_log)))
print('Best parameters: {}'.format(gs.best_params_))
print('Best cross-validation: {}'.format(gs.best_score_))


Test set score: 0.701688117080102
Best parameters: {'C': 10, 'kernel': 'rbf'}
Best cross-validation: 0.711107903019747


In [17]:
#グリッドサーチの結果を学習に適用する
svm_3 = SVR(gamma='scale',C=10, kernel='rbf') # SVCに訓練データを学習させる
svm_3.fit(X_train_log, y_train_log.values.ravel()) # SVCによる検証データの分類
svm_3_predict = svm_3.predict(X_test_log) #推定

In [18]:
blend_4 = (svm_3_predict.reshape(-1,1) + svm_2_predict.reshape(-1,1) + svm_1_predict.reshape(-1,1))/3
#mse
blend_4_mse = mean_squared_error(y_test_log, blend_4)
print('blend_4_mse', blend_4_mse)

blend_4_mse 0.06155771559672392


## 5. 決定木にグリッドサーチをかけ,パラメーターチューニングを行う。

In [19]:
param_grid = {"max_depth": [2,4,6,8,10],
              "min_samples_split": [2, 3, 5],
              "min_samples_leaf": [1,5,8]}

gs = GridSearchCV(estimator=DecisionTreeRegressor(),
                 param_grid = param_grid,   
                 scoring='neg_mean_squared_error', 
                  cv=5)

gs.fit(X_train_log, y_train_log)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeRegressor(criterion='mse', max_depth=None,
                                             max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort=False, random_state=None,
                                             splitter='best'),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': [2, 4, 6, 8, 10],
                         'min_samples_leaf': [1, 5, 8],
                         'min_samples_split': [2, 3, 5]},
             pre_dispatch='2*n_jobs', refi

In [20]:
#結果の出力
print('Test set score: {}'.format(gs.score(X_test_log, y_test_log)))
print('Best parameters: {}'.format(gs.best_params_))
print('Best cross-validation: {}'.format(gs.best_score_))


Test set score: -0.049533236634605675
Best parameters: {'max_depth': 6, 'min_samples_leaf': 8, 'min_samples_split': 3}
Best cross-validation: -0.04615085963671587


In [21]:
# 決定木(パラメーターチューニングしたものを使用)
dtr_1 = DecisionTreeRegressor(max_depth=6, min_samples_leaf=8, min_samples_split=2)
dtr_1.fit(X_train_log, y_train_log)#学習
dtr_1_predict = dtr_1.predict(X_test_log)#推定

In [22]:
blend_5 = (dtr_1_predict.reshape(-1, 1)+svm_1_predict.reshape(-1, 1)+lr_predict)/3
#mse
blend_5_mse = mean_squared_error(y_test_log, blend_5)
print("blend_5_mse", blend_5_mse)

blend_5_mse 0.04309929579367665


## 比較

In [23]:
#ブレンディングの精度
score_1 = pd.DataFrame([[blend_1_mse], 
                      [blend_2_mse], 
                      [blend_3_mse],
                     [blend_4_mse],
                     [blend_5_mse]], columns={'MSE'}, index=['平均をとる', '重み付け', 'SVMカーネル変更', 'GS_SVM', 'GS_決定木'])
display(score_1)
display(score)

,MSE
平均をとる,0.050547
重み付け,0.051327
SVMカーネル変更,0.061562
GS_SVM,0.061558
GS_決定木,0.043099


,単一モデルのMSE
SVM,0.052578
決定木,0.071384
線形回帰,0.044795


【考察】単一モデルでは線形回帰が最も低く、ブレンディングでは「平均をとったもの」、「重み付けをしたもの」、「グリッドサーチで決定木のパラメーターチューニングをしたもの」が低い値が出た。

# 【問題2】バギングのスクラッチ実装
バギング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。

バギングとは  
バギングは入力データの選び方を多様化する方法です。学習データから重複を許した上でランダムに抜き出すことで、N種類のサブセット（ ブートストラップサンプル ）を作り出します。それらによってモデルをN個学習し、推定結果の平均をとります。ブレンディングと異なり、それぞれの重み付けを変えることはありません。

sklearn.model_selection.train_test_split — scikit-learn 0.21.3 documentation

scikit-learnのtrain_test_splitを、shuffleパラメータをTrueにして使うことで、ランダムにデータを分割することができます。これによりブートストラップサンプルが手に入ります。

推定結果の平均をとる部分はブースティングと同様の実装になります。

## SVM

In [24]:
# #SVM
# svm = SVR(gamma='scale', C=1.0, epsilon=0.2) # SVCに訓練データを学習させる
# svm.fit(X_train_log, y_train_log) # SVCによる検証データの分類
# svm_predict = svm.predict(X_test_log) #推定
# svm_msxe = mean_squared_error(y_test_log, svm_predict) #mse
# print(svm_predict.shape)
# print(X_train.shape)

In [25]:
#SVM

#推定結果を格納するリスト
SVM_zero = np.zeros_like(y_test_log).ravel()
m = 30

for i in range(m):
    #学習データをランダムに抽出
    new_X_train, _, new_y_train, _ = train_test_split(X_train_log, y_train_log, shuffle=True, train_size=0.8)
    # SVMに訓練データを学習させる
    svm = SVR(gamma='scale', C=10, epsilon=0.2)
    # SVMによる検証データの分類
    svm.fit(new_X_train, new_y_train.values.ravel())
    #推定
    svm_predict = svm.predict(X_test_log)
    #リストに足し上げていく
    SVM_zero += svm_predict.ravel()

#推定結果の平均を出す
predict_mean = SVM_zero/m
y_test_log = y_test_log


#mse
mse_svm = mean_squared_error(y_test_log, predict_mean)
#print
    
print("バギング結果、MSE_svmは{:.6f}".format(mse_svm))    

バギング結果、MSE_svmは0.048992


## 線形回帰

In [26]:
#線形回帰

#推定結果を格納するリスト
lr_zero = np.zeros_like(y_test_log).ravel()
m = 10

for i in range(m):
    #学習データをランダムに抽出
    new_X_train, _, new_y_train, _ = train_test_split(X_train_log, y_train_log, shuffle=True, train_size=0.8)
    #訓練データを学習させる
    lr = LinearRegression()
    # SVMによる検証データの分類
    lr.fit(X_train_log, y_train_log)#学習
    #推定
    lr_predict = lr.predict(X_test_log)#推定

    #リストに足し上げていく
    lr_zero += lr_predict.ravel()

#推定結果の平均を出す
predict_mean_lr = lr_zero/m
#mse
mse_lr = mean_squared_error(y_test_log, predict_mean_lr)
#print
    
print("バギング結果、MSE_lrは{:.6f}".format(mse_lr))    

バギング結果、MSE_lrは0.044795


## 決定木

In [27]:
#決定木

#推定結果を格納するリスト
dtr_zero = np.zeros_like(y_test_log).ravel()
m = 10

for i in range(m):
    #学習データをランダムに抽出
    new_X_train, _, new_y_train, _ = train_test_split(X_train_log, y_train_log, shuffle=True, train_size=0.8)
    #訓練データを学習させる
    dtr = DecisionTreeRegressor(max_depth=6, min_samples_leaf=5)
    # SVMによる検証データの分類
    dtr.fit(X_train_log, y_train_log)#学習
    #推定
    dtr_predict = dtr.predict(X_test_log)#推定

    #リストに足し上げていく
    dtr_zero += dtr_predict.ravel()

#推定結果の平均を出す
predict_mean_dtr = dtr_zero/m
#mse
mse_dtr = mean_squared_error(y_test_log, predict_mean_dtr)
#print
    
print("バギング結果、MSE_dtrは{:.6f}".format(mse_dtr))

バギング結果、MSE_dtrは0.049163


## 比較

In [28]:
score_2 = pd.DataFrame([[mse_svm],
                        [mse_lr],
                        [mse_dtr]], columns=['バギングMSE'], index={'SVM', '線形回帰', '決定木'} )
display(score_2)
display(score)

,バギングMSE
SVM,0.048992
線形回帰,0.044795
決定木,0.049163


,単一モデルのMSE
SVM,0.052578
決定木,0.071384
線形回帰,0.044795


【考察】決定木とSVMでバギングを行ったものが単一モデルよりも精度が良かった。

# 【問題3】スタッキングのスクラッチ実装
スタッキング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。

スタッキングとは
スタッキングの手順は以下の通りです。最低限ステージ0とステージ1があればスタッキングは成立するため、それを実装してください。まずは 
K
0
=
3
,
M
0
=
2
 程度にします。

《学習時》

（ステージ 
0
 ）

学習データを 
K
0
 個に分割する。
分割した内の 
(
K
0
−
1
)
 個をまとめて学習用データ、残り 
1
 個を推定用データとする組み合わせが 
K
0
 個作れる。
あるモデルのインスタンスを 
K
0
 個用意し、異なる学習用データを使い学習する。
それぞれの学習済みモデルに対して、使っていない残り 
1
 個の推定用データを入力し、推定値を得る。（これをブレンドデータと呼ぶ）
さらに、異なるモデルのインスタンスも 
K
0
 個用意し、同様のことを行う。モデルが 
M
0
 個あれば、 
M
0
 個のブレンドデータが得られる。
（ステージ 
n
 ）

ステージ 
n
−
1
 のブレンドデータを
M
n
−
1
 次元の特徴量を持つ学習用データと考え、 
K
n
 個に分割する。以下同様である。
（ステージ 
N
 ）＊最後のステージ

ステージ 
N
−
1
 の 
M
N
−
1
 個のブレンドデータを
M
N
−
1
次元の特徴量の入力として、1種類のモデルの学習を行う。これが最終的な推定を行うモデルとなる。
《推定時》

（ステージ 
0
 ）

テストデータを 
K
0
×
M
0
 個の学習済みモデルに入力し、
K
0
×
M
0
 個の推定値を得る。これを 
K
0
 の軸で平均値を求め 
M
0
 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）
（ステージ 
n
 ）

ステージ 
n
−
1
 で得たブレンドテストを 
K
n
×
M
n
 個の学習済みモデルに入力し、
K
n
×
M
n
 個の推定値を得る。これを 
K
n
 の軸で平均値を求め 
M
0
 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）
（ステージ 
N
 ）＊最後のステージ

ステージ 
N
−
1
 で得たブレンドテストを学習済みモデルに入力し、推定値を得る。

In [29]:
# >>> import numpy as np
# >>> from sklearn.model_selection import KFold
# >>> X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
# >>> y = np.array([1, 2, 3, 4])
# >>> kf = KFold(n_splits=2)
# >>> kf.get_n_splits(X)
# 2
# >>> print(kf)  
# KFold(n_splits=2, random_state=None, shuffle=False)
# >>> for train_index, test_index in kf.split(X):
# ...    print("TRAIN:", train_index, "TEST:", test_index)
# ...    X_train, X_test = X[train_index], X[test_index]
# ...    y_train, y_test = y[train_index], y[test_index]
# TRAIN: [2 3] TEST: [0 1]

In [30]:
# 決定木
dtr = DecisionTreeRegressor(max_depth=3)
dtr.fit(X_train_log, y_train_log)#学習
dtr_predict = dtr.predict(X_test_log)#推定
dtr_mse = mean_squared_error(y_test_log, dtr_predict) #mse

In [31]:
#SVM
svm = SVR(gamma='scale', C=1.0, epsilon=0.2) # SVCに訓練データを学習させる
svm.fit(X_train_log, y_train_log.values.ravel()) # SVCによる検証データの分類
svm_predict = svm.predict(X_test_log) #推定
svm_mse = mean_squared_error(y_test_log, svm_predict) #mse

In [32]:
#線形回帰
lr = LinearRegression()
lr.fit(X_train_log, y_train_log)#学習
lr_predict = lr.predict(X_test_log)#推定
lr_mse = mean_squared_error(y_test_log, lr_predict) #mse

In [33]:
mylist = list(range(4))
print(mylist)
print(mylist[2])

[0, 1, 2, 3]
2


In [34]:
#学習データの分割数
n=4
#モデル数
m=2

# データを分割し、変数に格納
X_0 = np.array_split(X, n, 0)
y_0 = np.array_split(y, n, 0)
print(X_0)

for i in range(n):
    
    dtr = DecisionTreeRegressor(max_depth=3)
    dtr.fit(X_0[i], y_0[i])#学習
    dtr_predict = dtr.predict(X_0[i])#推定
    
    
    
    
    

for j in range(n):
    lr = LinearRegression()
    lr.fit(X[j], y[j])#学習
    lr_predict = lr.predict(X[-1])#推定
        

[     GrLivArea  YearBuilt
0         1710       2003
1         1262       1976
2         1786       2001
3         1717       1915
4         2198       2000
5         1362       1993
6         1694       2004
7         2090       1973
8         1774       1931
9         1077       1939
10        1040       1965
11        2324       2005
12         912       1962
13        1494       2006
14        1253       1960
15         854       1929
16        1004       1970
17        1296       1967
18        1114       2004
19        1339       1958
20        2376       2005
21        1108       1930
22        1795       2002
23        1060       1976
24        1060       1968
25        1600       2007
26         900       1951
27        1704       2007
28        1600       1957
29         520       1927
..         ...        ...
335       1786       1965
336       1922       2005
337       1536       2002
338       1621       1984
339       1215       1958
340       1908       2002
341        

KeyError: 0

In [ ]:
y.shape

## クラス

In [ ]:
class stacking():
    def fit():
    def predict():